This notebook shows how to use [CDD](https://www.inf.ethz.ch/personal/fukudak/cdd_home/) to compute controlled invariant sets for an hybrid system.
We consider the `cruise_control.jl` example of HybridSystems.jl which comes from [this paper](https://dl.acm.org/citation.cfm?id=2461378).

In [ ]:
include(Pkg.dir("HybridSystems", "examples", "cruise_control.jl"));

In [ ]:
const va = 15.6
const vb = 24.5
const vc = 29.5
const v = (va, vb, vc)
const U = 4.
const m0 = 500
const T = 2
const N = 10
const M = 1
const H = 0.8;

In [ ]:
macro _time(x)
    quote
        y = @timed $(esc(x))
        # y[1] is returned value
        # y[2] is time in seconds
        y[2]
    end
end

In [ ]:
function liftu(S, sys::HybridSystems.DiscreteLinearControlSystem)
    [sys.A sys.B] \ S
end
function new_constraint(hs, S, q, t)
    @assert source(hs, t) == q
    σ = symbol(hs, t)
    r = target(hs, t)
    ABset = liftu(S[1], hs.resetmaps[σ])
    project(ABset, 1:statedim(hs, q))
end
function new_constraints(hs, S, q)
    map(t -> new_constraint(hs, S, q, t), out_transitions(hs, q))
end
function add_hrep!(S, h::HalfSpace)
    if S ⊆ h # If S ⊆ h, then adding h will not change affect S
        false
    else
        push!(S, SimpleHRepresentation(reshape(h.a, 1, length(h.a)), [h.β]))
        true
    end
end
function add_constraint!(S, P)
    added = count(map(h -> add_hrep!(S, h), ineqs(P))) + count(map(h -> add_hrep!(S, h), eqs(P)))
    removehredundancy!(S)
    added
end
function add_constraints!(S::Polyhedron, Ps::Vector{<:Polyhedron})
    added = 0
    for P in Ps
        added += add_constraint!(S, P)
    end
    added
end
function set_iteration!(hs, S)
    Ps = map(q -> new_constraints(hs, S, q), states(hs))
    added = add_constraints!.(S, Ps)
    @show added
end
function iterate!(hs, S, nit)
    sum = 0.
    for i in 1:2
        @show i
        cur = @_time set_iteration!(hs, S);
        @show cur
        sum += cur
    end
    sum
end

In [ ]:
t = zeros(10)

In [ ]:
for m in 1:1
    hs = cruise_control_example(N, m, vmin = 5., v=(va, vb, vc), U=U, H=H, sym=false, m0=500);
    I0 = hs.invariants;
    @show nineqs(I0[1])
    I = copy(I0);
    @show m
    t[m] = iterate!(hs, I, 2)
    @show t[m]
end

In [ ]:
using Plots
pyplot()
_savefig(name) = Plots.savefig("/home/blegat/Dropbox/Research/Images/CruiseControl$name.png");

In [ ]:
MM = 1:10
te = [0.10468482971191406, 0.2330489158630371, 0.8677887916564941, 1.4519941806793213, 2.107150077819824, 3.5698659420013428, 4.80554986000061, 7.2393341064453125, 11.002917051315308, 15.073602199554443]
Plots.plot(MM, te, color=:blue, label="", xlabel = "Number of trailers", ylabel="Solve Time [s]", xticks=1:10)
Plots.scatter!(MM, te, color=:blue, label="Ellipsoidal : Full computation")
Plots.plot!(MM, t, color=:red, label="")
Plots.scatter!(MM, t, color=:red, label="Polyhedral : only 2 iterations")
_savefig("Benchmark")
Plots.plot!()